# Qwen3 Dataset Builder - Tạo Training Data theo Chuẩn Qwen3

Notebook này giúp xây dựng và chuẩn bị dataset cho training Qwen3/Qwen3VL model.

## Tính năng:
- 📝 Tạo dataset từ nhiều nguồn (JSON, CSV, conversation logs)
- ✅ Validate format theo chuẩn Qwen3
- 🔄 Convert các format khác nhau sang format Qwen3
- 📊 Thống kê và preview dataset
- 💾 Export dataset (JSON, Hugging Face format)
- 🎯 Hỗ trợ cả text-only và multimodal (text + image)

## Format chuẩn Qwen3:

```json
{
  "messages": [
    {"role": "user", "content": [{"type": "text", "text": "..."}]},
    {"role": "assistant", "content": [{"type": "text", "text": "..."}]}
  ]
}
```

Theo tài liệu: https://qwen.readthedocs.io/en/latest/


## 1. Cài đặt Dependencies


In [ ]:
# Cài đặt các thư viện cần thiết
# Theo tài liệu Qwen: https://qwen.readthedocs.io/en/latest/
%pip install -q "transformers>=4.41.0,<5.0.0"
%pip install -q datasets
%pip install -q pandas
%pip install -q pillow  # Cho image processing (nếu dùng Qwen3VL)
%pip install -q tqdm

print("✅ Dependencies đã được cài đặt!")


## 2. Import Libraries


In [ ]:
import json
import os
from typing import List, Dict, Any, Optional, Union
from pathlib import Path
import pandas as pd
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import re
from collections import Counter

print("✅ Libraries đã được import!")


## 3. Qwen3DatasetBuilder Class

Class chính để xây dựng dataset theo chuẩn Qwen3


In [ ]:
class Qwen3DatasetBuilder:
    """
    Dataset Builder cho Qwen3/Qwen3VL training
    Theo chuẩn Qwen3: https://qwen.readthedocs.io/en/latest/
    
    Format chuẩn Qwen3:
    {
        "messages": [
            {"role": "user", "content": [{"type": "text", "text": "..."}]},
            {"role": "assistant", "content": [{"type": "text", "text": "..."}]}
        ]
    }
    
    Hoặc với image (Qwen3VL):
    {
        "messages": [
            {"role": "user", "content": [
                {"type": "image", "image": "path_or_url"},
                {"type": "text", "text": "..."}
            ]},
            {"role": "assistant", "content": [{"type": "text", "text": "..."}]}
        ]
    }
    """
    
    def __init__(self, include_image: bool = False):
        """
        Args:
            include_image: Nếu True, hỗ trợ image trong content (Qwen3VL)
        """
        self.include_image = include_image
        self.dataset = []
        self.stats = {
            "total_samples": 0,
            "total_conversations": 0,
            "total_turns": 0,
            "avg_turns_per_conv": 0,
            "total_tokens_estimate": 0,
        }
    
    def add_conversation(self, messages: List[Dict[str, Any]]) -> None:
        """
        Thêm một conversation vào dataset
        
        Args:
            messages: List of messages theo format Qwen3
                [
                    {"role": "user", "content": [{"type": "text", "text": "..."}]},
                    {"role": "assistant", "content": [{"type": "text", "text": "..."}]}
                ]
        """
        # Validate messages
        if not self._validate_messages(messages):
            raise ValueError("Messages không đúng format Qwen3")
        
        self.dataset.append({"messages": messages})
        self._update_stats(messages)
    
    def add_simple_qa(self, question: str, answer: str, image_path: Optional[str] = None) -> None:
        """
        Thêm một Q&A đơn giản (1 turn conversation)
        
        Args:
            question: Câu hỏi của user
            answer: Câu trả lời của assistant
            image_path: Đường dẫn ảnh (nếu có, cho Qwen3VL)
        """
        user_content = []
        
        # Thêm image nếu có
        if image_path and self.include_image:
            user_content.append({"type": "image", "image": image_path})
        
        # Thêm text
        user_content.append({"type": "text", "text": question})
        
        messages = [
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": [{"type": "text", "text": answer}]}
        ]
        
        self.add_conversation(messages)
    
    def add_multi_turn_conversation(self, turns: List[Dict[str, str]], image_path: Optional[str] = None) -> None:
        """
        Thêm multi-turn conversation
        
        Args:
            turns: List of turns, mỗi turn là {"user": "...", "assistant": "..."}
            image_path: Đường dẫn ảnh (nếu có, chỉ thêm vào turn đầu tiên)
        """
        messages = []
        
        for i, turn in enumerate(turns):
            if "user" not in turn or "assistant" not in turn:
                raise ValueError(f"Turn {i} phải có 'user' và 'assistant'")
            
            # User message
            user_content = []
            if image_path and self.include_image and i == 0:  # Chỉ thêm image vào turn đầu
                user_content.append({"type": "image", "image": image_path})
            user_content.append({"type": "text", "text": turn["user"]})
            messages.append({"role": "user", "content": user_content})
            
            # Assistant message
            messages.append({
                "role": "assistant",
                "content": [{"type": "text", "text": turn["assistant"]}]
            })
        
        self.add_conversation(messages)
    
    def load_from_json(self, file_path: str, format_type: str = "auto") -> None:
        """
        Load dataset từ JSON file
        
        Args:
            file_path: Đường dẫn file JSON
            format_type: Format của file ("auto", "qwen3", "qa", "conversation")
        """
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if format_type == "auto":
            format_type = self._detect_format(data)
        
        if format_type == "qwen3":
            # Format đã đúng Qwen3
            if isinstance(data, list):
                for item in data:
                    if "messages" in item:
                        self.add_conversation(item["messages"])
            elif isinstance(data, dict) and "messages" in data:
                self.add_conversation(data["messages"])
        elif format_type == "qa":
            # Format Q&A: [{"question": "...", "answer": "..."}]
            for item in data:
                self.add_simple_qa(item.get("question", ""), item.get("answer", ""))
        elif format_type == "conversation":
            # Format conversation: [{"turns": [{"user": "...", "assistant": "..."}]}]
            for item in data:
                if "turns" in item:
                    self.add_multi_turn_conversation(item["turns"])
        
        print(f"✅ Đã load {len(data) if isinstance(data, list) else 1} samples từ {file_path}")
    
    def load_from_csv(self, file_path: str, question_col: str = "question", answer_col: str = "answer") -> None:
        """
        Load dataset từ CSV file
        
        Args:
            file_path: Đường dẫn file CSV
            question_col: Tên cột chứa câu hỏi
            answer_col: Tên cột chứa câu trả lời
        """
        df = pd.read_csv(file_path)
        
        if question_col not in df.columns or answer_col not in df.columns:
            raise ValueError(f"CSV phải có columns: {question_col}, {answer_col}")
        
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Loading CSV"):
            question = str(row[question_col]).strip()
            answer = str(row[answer_col]).strip()
            
            if question and answer:
                self.add_simple_qa(question, answer)
        
        print(f"✅ Đã load {len(df)} samples từ {file_path}")
    
    def load_from_conversation_logs(self, file_path: str) -> None:
        """
        Load từ conversation logs (format: mỗi dòng là một JSON conversation)
        
        Args:
            file_path: Đường dẫn file (JSONL format)
        """
        count = 0
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                if not line:
                    continue
                
                try:
                    data = json.loads(line)
                    if "messages" in data:
                        self.add_conversation(data["messages"])
                        count += 1
                except json.JSONDecodeError as e:
                    print(f"⚠️  Lỗi ở dòng {line_num}: {e}")
        
        print(f"✅ Đã load {count} samples từ {file_path}")
    
    def _validate_messages(self, messages: List[Dict[str, Any]]) -> bool:
        """
        Validate messages format theo chuẩn Qwen3
        """
        if not isinstance(messages, list) or len(messages) == 0:
            return False
        
        for msg in messages:
            if not isinstance(msg, dict):
                return False
            
            if "role" not in msg or "content" not in msg:
                return False
            
            if msg["role"] not in ["user", "assistant", "system"]:
                return False
            
            if not isinstance(msg["content"], list):
                return False
            
            for content_item in msg["content"]:
                if not isinstance(content_item, dict):
                    return False
                if "type" not in content_item:
                    return False
                if content_item["type"] == "text":
                    if "text" not in content_item:
                        return False
                elif content_item["type"] == "image":
                    if "image" not in content_item:
                        return False
                    if not self.include_image:
                        return False  # Không hỗ trợ image
                else:
                    return False  # Type không hợp lệ
        
        return True
    
    def _detect_format(self, data: Any) -> str:
        """
        Tự động detect format của data
        """
        if isinstance(data, list) and len(data) > 0:
            first_item = data[0]
            if isinstance(first_item, dict):
                if "messages" in first_item:
                    return "qwen3"
                elif "question" in first_item and "answer" in first_item:
                    return "qa"
                elif "turns" in first_item:
                    return "conversation"
        elif isinstance(data, dict):
            if "messages" in data:
                return "qwen3"
        
        return "qa"  # Default
    
    def _update_stats(self, messages: List[Dict[str, Any]]) -> None:
        """
        Cập nhật thống kê
        """
        self.stats["total_conversations"] += 1
        self.stats["total_turns"] += len([m for m in messages if m["role"] == "user"])
        
        # Estimate tokens (rough: ~4 chars per token)
        total_chars = 0
        for msg in messages:
            for content in msg.get("content", []):
                if content.get("type") == "text":
                    total_chars += len(content.get("text", ""))
        self.stats["total_tokens_estimate"] += total_chars // 4
        
        self.stats["total_samples"] = len(self.dataset)
        if self.stats["total_conversations"] > 0:
            self.stats["avg_turns_per_conv"] = self.stats["total_turns"] / self.stats["total_conversations"]
    
    def get_stats(self) -> Dict[str, Any]:
        """
        Lấy thống kê dataset
        """
        return self.stats.copy()
    
    def preview(self, n: int = 3) -> None:
        """
        Preview n samples đầu tiên
        """
        print(f"\n📊 Preview {min(n, len(self.dataset))} samples đầu tiên:\n")
        for i, sample in enumerate(self.dataset[:n], 1):
            print(f"--- Sample {i} ---")
            print(json.dumps(sample, ensure_ascii=False, indent=2))
            print()
    
    def to_huggingface_dataset(self) -> Dataset:
        """
        Convert sang Hugging Face Dataset
        """
        return Dataset.from_list(self.dataset)
    
    def save_to_json(self, file_path: str, indent: int = 2) -> None:
        """
        Lưu dataset ra JSON file
        """
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(self.dataset, f, ensure_ascii=False, indent=indent)
        print(f"✅ Đã lưu {len(self.dataset)} samples vào {file_path}")
    
    def save_to_jsonl(self, file_path: str) -> None:
        """
        Lưu dataset ra JSONL file (mỗi dòng một JSON)
        """
        with open(file_path, 'w', encoding='utf-8') as f:
            for item in self.dataset:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        print(f"✅ Đã lưu {len(self.dataset)} samples vào {file_path}")
    
    def split_train_test(self, test_ratio: float = 0.1, shuffle: bool = True, seed: int = 42) -> DatasetDict:
        """
        Chia dataset thành train và test
        
        Args:
            test_ratio: Tỷ lệ test set (0.0 - 1.0)
            shuffle: Có shuffle không
            seed: Random seed
        """
        dataset = self.to_huggingface_dataset()
        
        if shuffle:
            dataset = dataset.shuffle(seed=seed)
        
        split_dataset = dataset.train_test_split(test_size=test_ratio, seed=seed)
        
        return split_dataset

print("✅ Qwen3DatasetBuilder class đã được định nghĩa!")


## 4. Ví dụ sử dụng - Tạo Dataset từ Q&A đơn giản


In [ ]:
# Tạo builder (text-only, không có image)
builder = Qwen3DatasetBuilder(include_image=False)

# Thêm một số Q&A mẫu về ngân hàng
builder.add_simple_qa(
    question="Lãi suất tiết kiệm hiện tại là bao nhiêu?",
    answer="Lãi suất tiết kiệm hiện tại dao động từ 4.5% đến 6.5% tùy theo kỳ hạn và loại sản phẩm. Bạn có thể liên hệ chi nhánh gần nhất để biết thêm chi tiết."
)

builder.add_simple_qa(
    question="Làm thế nào để mở tài khoản ngân hàng?",
    answer="Để mở tài khoản ngân hàng, bạn cần: 1) CMND/CCCD, 2) Đến chi nhánh hoặc đăng ký online, 3) Điền form đăng ký, 4) Nộp phí mở tài khoản (nếu có). Thời gian xử lý thường 1-2 ngày làm việc."
)

builder.add_simple_qa(
    question="Tôi có thể vay tín chấp với mức lương 10 triệu không?",
    answer="Với mức lương 10 triệu, bạn có thể vay tín chấp với hạn mức khoảng 50-100 triệu tùy thuộc vào điểm tín dụng và điều kiện của ngân hàng. Lãi suất thường từ 15-25%/năm."
)

# Preview dataset
builder.preview()

# Xem thống kê
stats = builder.get_stats()
print("\n📊 Thống kê:")
for key, value in stats.items():
    print(f"  {key}: {value}")


## 5. Ví dụ - Multi-turn Conversation


In [ ]:
# Thêm multi-turn conversation
builder.add_multi_turn_conversation([
    {
        "user": "Tôi muốn biết về sản phẩm tiết kiệm",
        "assistant": "Chúng tôi có nhiều sản phẩm tiết kiệm với các kỳ hạn khác nhau. Bạn quan tâm đến kỳ hạn nào?"
    },
    {
        "user": "Kỳ hạn 6 tháng",
        "assistant": "Với kỳ hạn 6 tháng, lãi suất hiện tại là 5.2%/năm. Số tiền tối thiểu là 1 triệu đồng. Bạn có muốn mở tài khoản không?"
    },
    {
        "user": "Có, tôi muốn mở",
        "assistant": "Để mở tài khoản tiết kiệm, bạn cần đến chi nhánh với CMND/CCCD và số tiền gửi. Hoặc bạn có thể đăng ký online qua ứng dụng ngân hàng."
    }
])

print("✅ Đã thêm multi-turn conversation")
builder.preview(1)


## 6. Load từ JSON File


In [ ]:
# Tạo file JSON mẫu
sample_data = [
    {
        "question": "Thẻ tín dụng có phí thường niên không?",
        "answer": "Có, thẻ tín dụng thường có phí thường niên từ 200.000 - 2.000.000 đồng tùy loại thẻ. Một số ngân hàng miễn phí năm đầu hoặc miễn phí nếu đạt điều kiện chi tiêu."
    },
    {
        "question": "Làm sao để tăng hạn mức thẻ tín dụng?",
        "answer": "Để tăng hạn mức thẻ tín dụng, bạn cần: 1) Sử dụng thẻ thường xuyên và thanh toán đúng hạn, 2) Có thu nhập ổn định, 3) Đăng ký tăng hạn mức qua ứng dụng hoặc chi nhánh. Ngân hàng sẽ xem xét và quyết định."
    }
]

# Lưu vào file tạm
temp_json = "/tmp/sample_data.json"
with open(temp_json, 'w', encoding='utf-8') as f:
    json.dump(sample_data, f, ensure_ascii=False, indent=2)

# Load từ JSON
builder2 = Qwen3DatasetBuilder(include_image=False)
builder2.load_from_json(temp_json, format_type="qa")

print(f"\n✅ Đã load {len(builder2.dataset)} samples")
builder2.preview()


## 7. Load từ CSV File


In [ ]:
# Tạo file CSV mẫu
sample_csv_data = {
    "question": [
        "Tôi có thể chuyển tiền online không?",
        "Phí chuyển tiền là bao nhiêu?",
        "Thời gian chuyển tiền trong nước mất bao lâu?"
    ],
    "answer": [
        "Có, bạn có thể chuyển tiền online qua ứng dụng ngân hàng, internet banking hoặc ATM. Chỉ cần có tài khoản và đã đăng ký dịch vụ.",
        "Phí chuyển tiền trong nước thường từ 5.000 - 11.000 đồng/giao dịch tùy ngân hàng. Một số ngân hàng miễn phí một số giao dịch đầu tiên trong tháng.",
        "Chuyển tiền trong nước thường được xử lý ngay lập tức (real-time) hoặc trong vòng 1-2 giờ làm việc tùy ngân hàng và thời điểm giao dịch."
    ]
}

temp_csv = "/tmp/sample_data.csv"
pd.DataFrame(sample_csv_data).to_csv(temp_csv, index=False)

# Load từ CSV
builder3 = Qwen3DatasetBuilder(include_image=False)
builder3.load_from_csv(temp_csv, question_col="question", answer_col="answer")

print(f"\n✅ Đã load {len(builder3.dataset)} samples")
builder3.preview()


## 8. Validate và Export Dataset


In [ ]:
# Merge tất cả datasets
final_builder = Qwen3DatasetBuilder(include_image=False)

# Thêm từ builder1, builder2, builder3
for sample in builder.dataset:
    final_builder.add_conversation(sample["messages"])
for sample in builder2.dataset:
    final_builder.add_conversation(sample["messages"])
for sample in builder3.dataset:
    final_builder.add_conversation(sample["messages"])

# Thống kê cuối cùng
stats = final_builder.get_stats()
print("\n" + "="*50)
print("📊 THỐNG KÊ DATASET CUỐI CÙNG")
print("="*50)
for key, value in stats.items():
    print(f"{key}: {value}")

# Preview
print("\n" + "="*50)
print("PREVIEW DATASET")
print("="*50)
final_builder.preview(2)


## 9. Export Dataset


In [ ]:
# Export ra JSON
output_json = "./qwen3_training_dataset.json"
final_builder.save_to_json(output_json)

# Export ra JSONL (cho Hugging Face)
output_jsonl = "./qwen3_training_dataset.jsonl"
final_builder.save_to_jsonl(output_jsonl)

# Convert sang Hugging Face Dataset
hf_dataset = final_builder.to_huggingface_dataset()
print(f"\n✅ Hugging Face Dataset: {len(hf_dataset)} samples")
print(f"   Features: {hf_dataset.features}")

# Chia train/test
split_dataset = final_builder.split_train_test(test_ratio=0.2, shuffle=True, seed=42)
print(f"\n✅ Train set: {len(split_dataset['train'])} samples")
print(f"✅ Test set: {len(split_dataset['test'])} samples")


## 10. Upload lên Hugging Face (Optional)


In [ ]:
# Uncomment để upload dataset lên Hugging Face
# from huggingface_hub import login, HfApi
# 
# # Login
# login(token="YOUR_HF_TOKEN")
# 
# # Upload dataset
# dataset_id = "your-username/qwen3-bank-dataset"
# split_dataset.push_to_hub(dataset_id)
# print(f"✅ Đã upload dataset lên {dataset_id}")


## 11. Hướng dẫn sử dụng

### Format Dataset chuẩn Qwen3:

```json
{
  "messages": [
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Câu hỏi của người dùng"}
      ]
    },
    {
      "role": "assistant", 
      "content": [
        {"type": "text", "text": "Câu trả lời của AI"}
      ]
    }
  ]
}
```

### Format với Image (Qwen3VL):

```json
{
  "messages": [
    {
      "role": "user",
      "content": [
        {"type": "image", "image": "path_or_url"},
        {"type": "text", "text": "Mô tả ảnh này"}
      ]
    },
    {
      "role": "assistant",
      "content": [
        {"type": "text", "text": "Mô tả về ảnh..."}
      ]
    }
  ]
}
```

### Các cách tạo dataset:

1. **Từ Q&A đơn giản**: `builder.add_simple_qa(question, answer)`
2. **Từ multi-turn conversation**: `builder.add_multi_turn_conversation(turns)`
3. **Từ JSON file**: `builder.load_from_json(file_path)`
4. **Từ CSV file**: `builder.load_from_csv(file_path)`
5. **Từ conversation logs**: `builder.load_from_conversation_logs(file_path)`

### Best Practices:

- ✅ Đảm bảo mỗi conversation có ít nhất user + assistant message
- ✅ Validate format trước khi thêm vào dataset
- ✅ Sử dụng JSONL format cho dataset lớn (dễ xử lý từng dòng)
- ✅ Chia train/test để đánh giá model
- ✅ Preview dataset trước khi export

### Tài liệu tham khảo:

Theo [Qwen Documentation](https://qwen.readthedocs.io/en/latest/):
- Format messages phải đúng chuẩn Qwen3
- Hỗ trợ cả text-only và multimodal (text + image)
- Dataset có thể upload lên Hugging Face để chia sẻ
